In [1]:
import os
import torch
from dataloader import MRDataset
from tqdm import tqdm
from tqdm import tqdm_notebook
from torchsample.transforms import RandomRotate, RandomTranslate, RandomFlip, ToTensor, Compose, RandomAffine
import torch.nn as nn
from torch.utils import data
from torchvision import transforms
from torchvision import datasets
import matplotlib.pyplot as plt
import numpy as np
from torch.nn import functional as F
import io
import requests
from PIL import Image
from torchvision import models, transforms
from torch.autograd import Variable
import cv2
import pdb
import model
import shutil

In [26]:

m =torch.load('models/model_naa_acl_sagittal_val_auc_0.8774_train_auc_0.8531_epoch_17.pth')

m.eval()

# hook the feature extractor
features_blobs = []
def hook_feature(module, input, output):
    features_blobs.append(output.data.cpu().numpy())

m._modules.get('pretrained_model').features.register_forward_hook(hook_feature);

In [2]:
def returnCAM(feature_conv, weight_softmax, class_idx):
    size_upsample = (256, 256)
    bz, nc, h, w = feature_conv.shape
    slice_cams = []
    for s in range(bz):
        for idx in class_idx:
            cam = weight_softmax[idx].dot(feature_conv[s].reshape((nc, h*w)))
            cam = cam.reshape(h, w)
            cam = cam - np.min(cam)
            cam_img = cam / np.max(cam)
            cam_img = np.uint8(255 * cam_img)
            slice_cams.append(cv2.resize(cam_img, size_upsample))
    return slice_cams

In [27]:


#for the purposes of grad-cam, turn off the transform and shuffling
train_dataset = MRDataset('./data/', 'acl',
                              'sagittal', transform=None, train=True)

train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=1, shuffle=False, num_workers=11, drop_last=False)

In [29]:
patients = []
for i, (image, label, _) in tqdm_notebook(enumerate(train_loader), total=len(train_loader)):
    patient_data = {}
    patient_data['mri'] = image
    patient_data['label'] = label
    patient_data['id'] = '0' * (4 - len(str(i))) + str(i)
    patients.append(patient_data)


acl = list(filter(lambda d: d['label'] == 1, patients))

<ipython-input-29-09fa821410f0>:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i, (image, label, _) in tqdm_notebook(enumerate(train_loader), total=len(train_loader)):


0


<ipython-input-30-9875299eb29e>:30: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for s in tqdm_notebook(range(num_slices), leave=False):


1


2


3


...

5


6


7


8


9


In [12]:
acl[1]['id']

'0018'

In [17]:
for i in range(0,100):
    print(i)
    case  = acl[i]

    patient_id = case['id']
    mri = case['mri']

    plane = 'sagittal'
    folder_path = f'./grad_cams/{plane}/{patient_id}/'
    if os.path.isdir(folder_path):
        shutil.rmtree(folder_path)
    os.makedirs(folder_path)
    os.makedirs(folder_path + 'slices/')
    os.makedirs(folder_path + 'cams/')

    params = list(m.parameters())
    weight_softmax = np.squeeze(params[-2].cpu().data.numpy())

    num_slices = mri.shape[1]
    logit = m(mri)
    size_upsample = (256, 256)

    h_x = F.softmax(logit, dim=1).data.squeeze(0)
    probs, idx = h_x.sort(0, True)
    probs = probs.cpu().numpy()
    idx = idx.cpu().numpy()
    slice_cams = returnCAM(features_blobs[-1], weight_softmax, idx[:1])
    
    for s in tqdm_notebook(range(num_slices), leave=False):
        slice_pil = (transforms
                     .ToPILImage()(mri.cpu()[0][s] / 255))
        slice_pil.save(folder_path + f'slices/{s}.png', 
                       dpi=(300, 300))
         
        img = mri[0][s].cpu().numpy()
        img = img.transpose(1, 2, 0)
        heatmap = (cv2
                    .cvtColor(cv2.applyColorMap(
                        cv2.resize(slice_cams[s], (256, 256)),
                        cv2.COLORMAP_JET), 
                               cv2.COLOR_BGR2RGB)
                  )
        result = heatmap * 0.3 + img * 0.5  
        
        pil_img_cam = Image.fromarray(np.uint8(result))
        pil_img_cam.save(folder_path + f'cams/{s}.png', dpi=(300, 300))

0


<ipython-input-17-35e613d73050>:29: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for s in tqdm_notebook(range(num_slices), leave=False):


1


2


3


...

95


96


97


98


99


## coronal

In [3]:

m =torch.load('models/model_naa_acl_coronal_val_auc_0.8504_train_auc_0.8610_epoch_29.pth')

m.eval()

# hook the feature extractor
features_blobs = []
def hook_feature(module, input, output):
    features_blobs.append(output.data.cpu().numpy())

m._modules.get('pretrained_model').features.register_forward_hook(hook_feature);

In [4]:


#for the purposes of grad-cam, turn off the transform and shuffling
train_dataset = MRDataset('./data/', 'acl',
                              'coronal', transform=None, train=True)

train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=1, shuffle=False, num_workers=11, drop_last=False)

In [5]:
patients = []
for i, (image, label, _) in tqdm_notebook(enumerate(train_loader), total=len(train_loader)):
    patient_data = {}
    patient_data['mri'] = image
    patient_data['label'] = label
    patient_data['id'] = '0' * (4 - len(str(i))) + str(i)
    patients.append(patient_data)


acl = list(filter(lambda d: d['label'] == 1, patients))

<ipython-input-5-09fa821410f0>:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i, (image, label, _) in tqdm_notebook(enumerate(train_loader), total=len(train_loader)):


0


<ipython-input-7-9875299eb29e>:30: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for s in tqdm_notebook(range(num_slices), leave=False):


1


2


3


...

5


6


7


8


9


In [11]:
def generate_cams(num, plane):
    for i in range(0,num):
        print(i)
        case  = acl[i]

        patient_id = case['id']
        mri = case['mri']

        plane = plane
        folder_path = f'./grad_cams/{plane}/{patient_id}/'
        if os.path.isdir(folder_path):
            shutil.rmtree(folder_path)
        os.makedirs(folder_path)
        os.makedirs(folder_path + 'slices/')
        os.makedirs(folder_path + 'cams/')

        params = list(m.parameters())
        weight_softmax = np.squeeze(params[-2].cpu().data.numpy())

        num_slices = mri.shape[1]
        logit = m(mri)
        size_upsample = (256, 256)

        h_x = F.softmax(logit, dim=1).data.squeeze(0)
        probs, idx = h_x.sort(0, True)
        probs = probs.cpu().numpy()
        idx = idx.cpu().numpy()
        slice_cams = returnCAM(features_blobs[-1], weight_softmax, idx[:1])

        for s in tqdm_notebook(range(num_slices), leave=False):
            slice_pil = (transforms
                         .ToPILImage()(mri.cpu()[0][s] / 255))
            slice_pil.save(folder_path + f'slices/{s}.png', 
                           dpi=(300, 300))

            img = mri[0][s].cpu().numpy()
            img = img.transpose(1, 2, 0)
            heatmap = (cv2
                        .cvtColor(cv2.applyColorMap(
                            cv2.resize(slice_cams[s], (256, 256)),
                            cv2.COLORMAP_JET), 
                                   cv2.COLOR_BGR2RGB)
                      )
            result = heatmap * 0.3 + img * 0.5  

            pil_img_cam = Image.fromarray(np.uint8(result))
            pil_img_cam.save(folder_path + f'cams/{s}.png', dpi=(300, 300))

In [22]:
generate_cams(10,'coronal')

0


<ipython-input-21-2ed70385254b>:30: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for s in tqdm_notebook(range(num_slices), leave=False):


1


2


3


...

5


6


7


8


9


## axial

In [4]:

m =torch.load('models/model_naa_acl_axial_val_auc_0.9574_train_auc_0.9393_epoch_32.pth')

m.eval()

# hook the feature extractor
features_blobs = []
def hook_feature(module, input, output):
    features_blobs.append(output.data.cpu().numpy())

m._modules.get('pretrained_model').features.register_forward_hook(hook_feature);

In [8]:


#for the purposes of grad-cam, turn off the transform and shuffling
train_dataset = MRDataset('./data/', 'acl',
                              'axial', transform=None, train=True)

train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=1, shuffle=False, num_workers=11, drop_last=False)

In [9]:
patients = []
for i, (image, label, _) in tqdm_notebook(enumerate(train_loader), total=len(train_loader)):
    patient_data = {}
    patient_data['mri'] = image
    patient_data['label'] = label
    patient_data['id'] = '0' * (4 - len(str(i))) + str(i)
    patients.append(patient_data)


acl = list(filter(lambda d: d['label'] == 1, patients))

<ipython-input-9-473d9097aa14>:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i, (image, label, _) in tqdm_notebook(enumerate(train_loader), total=len(train_loader)):


In [13]:
generate_cams(10,'axial')

0


<ipython-input-11-9875299eb29e>:30: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for s in tqdm_notebook(range(num_slices), leave=False):


1


2


3


...

5


6


7


8


9
